In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_8/models/B5R2b5_5FP_1FC_fold8_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image4/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold4.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold4.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold8.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(656, 26)
(656, 26)
Normal:  (428, 26)
Abnormal:  (228, 26)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,215,215,20,P3,P31,Abnormal,AB01,...,152.0,96.0,617.0,548.0,0.153985,0.107041,0.623188,0.609562,AB01 P3-1 C020.JPG,AB01 P3-1 C020.JPG
1,1,1,1,216,216,20,P2,P2,Abnormal,AB01,...,233.0,119.0,568.0,512.0,0.234657,0.132937,0.572202,0.569721,AB01 P2 C020.JPG,AB01 P2 C020.JPG
2,2,2,2,217,217,20,P5,P51,Abnormal,AB01,...,213.0,135.0,568.0,640.0,0.215190,0.150865,0.573237,0.711155,AB01 P5-1 C020.JPG,AB01 P5-1 C020.JPG
3,3,3,3,218,218,20,P1,P1,Abnormal,AB01,...,186.0,65.0,593.0,541.0,0.185053,0.073176,0.588968,0.601594,AB01 P1 C020.JPG,AB01 P1 C020.JPG
4,4,4,4,219,219,20,P5,P52,Abnormal,AB01,...,80.0,132.0,590.0,491.0,0.081967,0.146881,0.599271,0.545817,AB01 P5-2 C020.JPG,AB01 P5-2 C020.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 656 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 1.19803166, -0.20051649, -0.20429447, ...,  0.14328043,
        -0.08281653,  0.71018374],
       [-0.06959179,  0.32978576,  0.3009896 , ..., -0.12592344,
        -0.09477881,  0.12283552],
       [-0.16375944, -0.19546986,  0.1917129 , ...,  0.4020023 ,
         0.83446562, -0.16819173],
       ...,
       [-0.10911893, -0.20846185, -0.16113017, ...,  0.42430159,
        -0.01692413,  0.0623855 ],
       [ 0.1927506 ,  0.22931138,  0.0788432 , ..., -0.07260881,
         0.01758362,  0.03312831],
       [ 0.53767151,  0.86481857,  0.35018721, ..., -0.14041914,
         0.15952471,  0.8355186 ]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.198032,-0.200516,-0.204294,-0.098968,0.135859,0.004413,-0.184289,0.786509,0.184650,0.672510,...,0.443565,-0.114890,-0.079841,0.710071,-0.196895,1.102819,0.600976,0.143280,-0.082817,0.710184
1,-0.069592,0.329786,0.300990,0.388246,0.010353,0.018634,0.064608,0.151934,0.235241,-0.007014,...,0.124506,-0.147122,-0.170247,-0.153701,-0.158448,-0.046067,-0.019178,-0.125923,-0.094779,0.122836
2,-0.163759,-0.195470,0.191713,0.411987,0.244497,-0.056774,0.130079,-0.111927,0.434972,-0.132526,...,0.214111,0.241133,0.948537,-0.050538,0.614593,-0.084680,-0.047273,0.402002,0.834466,-0.168192
3,0.289818,0.765759,0.541988,0.247122,0.225844,-0.130515,0.589063,0.354398,-0.116416,-0.191735,...,0.278303,-0.167865,-0.227566,-0.178349,-0.163193,0.100586,0.085105,-0.170757,-0.038185,0.367311
4,-0.068669,-0.013692,0.440784,0.404373,0.197310,-0.138683,0.409092,-0.118299,0.761165,-0.086374,...,0.343699,0.389163,0.773190,-0.150706,0.881634,-0.028804,0.063383,-0.002162,0.345757,-0.098269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0.135851,-0.228165,-0.031476,-0.103109,-0.130443,-0.133419,-0.012839,-0.168552,-0.204619,-0.237604,...,-0.073047,0.418648,-0.200451,0.056706,0.152588,0.021057,-0.216978,-0.147375,-0.225492,-0.196037
652,0.232987,-0.068043,0.381478,0.060085,-0.170231,0.096958,0.690560,-0.239708,-0.162655,-0.127625,...,0.005665,0.574592,-0.035191,0.365886,0.602972,0.016599,-0.119977,-0.118134,-0.176198,-0.129231
653,-0.109119,-0.208462,-0.161130,-0.058967,-0.074887,-0.152327,-0.137089,0.045530,-0.099612,0.153849,...,-0.024655,-0.105445,-0.149885,0.182731,-0.217071,0.158556,0.072848,0.424302,-0.016924,0.062385
654,0.192751,0.229311,0.078843,0.362329,0.242724,-0.061710,0.174097,0.587090,0.244477,0.091040,...,0.525322,-0.174046,-0.120564,-0.198428,-0.088734,0.371496,0.328564,-0.072609,0.017584,0.033128


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,1.198032,-0.200516,-0.204294,-0.098968,0.135859,0.004413,-0.184289,0.786509,0.184650,0.672510,...,-0.196895,1.102819,0.600976,0.143280,-0.082817,0.710184,Abnormal,P31,FP-B,AB01
1,-0.069592,0.329786,0.300990,0.388246,0.010353,0.018634,0.064608,0.151934,0.235241,-0.007014,...,-0.158448,-0.046067,-0.019178,-0.125923,-0.094779,0.122836,Abnormal,P2,FP-A,AB01
2,-0.163759,-0.195470,0.191713,0.411987,0.244497,-0.056774,0.130079,-0.111927,0.434972,-0.132526,...,0.614593,-0.084680,-0.047273,0.402002,0.834466,-0.168192,Abnormal,P51,FP-C,AB01
3,0.289818,0.765759,0.541988,0.247122,0.225844,-0.130515,0.589063,0.354398,-0.116416,-0.191735,...,-0.163193,0.100586,0.085105,-0.170757,-0.038185,0.367311,Abnormal,P1,FP-A,AB01
4,-0.068669,-0.013692,0.440784,0.404373,0.197310,-0.138683,0.409092,-0.118299,0.761165,-0.086374,...,0.881634,-0.028804,0.063383,-0.002162,0.345757,-0.098269,Abnormal,P52,FP-C,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0.135851,-0.228165,-0.031476,-0.103109,-0.130443,-0.133419,-0.012839,-0.168552,-0.204619,-0.237604,...,0.152588,0.021057,-0.216978,-0.147375,-0.225492,-0.196037,Normal,P71,FP-D,Normal
652,0.232987,-0.068043,0.381478,0.060085,-0.170231,0.096958,0.690560,-0.239708,-0.162655,-0.127625,...,0.602972,0.016599,-0.119977,-0.118134,-0.176198,-0.129231,Normal,P72,FP-D,Normal
653,-0.109119,-0.208462,-0.161130,-0.058967,-0.074887,-0.152327,-0.137089,0.045530,-0.099612,0.153849,...,-0.217071,0.158556,0.072848,0.424302,-0.016924,0.062385,Normal,P8,FP-E,Normal
654,0.192751,0.229311,0.078843,0.362329,0.242724,-0.061710,0.174097,0.587090,0.244477,0.091040,...,-0.088734,0.371496,0.328564,-0.072609,0.017584,0.033128,Normal,P1,FP-A,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_5FP_RF_5FP_fold8_1.csv')